In [675]:
#!pip install pandas
#!pip install numpy
#!pip install rapidfuzz

In [676]:
import pandas as pd
import numpy as np

pd.set_option('display.float_format', '{:.2f}'.format)

Census data sourced from this site
https://www.census.gov/data/tables/time-series/demo/popest/2020s-metro-and-micro-statistical-areas-detail.html
MSA data dictionary available here
https://www2.census.gov/programs-surveys/popest/technical-documentation/file-layouts/2020-2024/CBSA-EST2024-ALLDATA-CHAR.pdf

In [677]:
df_sub = pd.read_csv('data/sub-est2024.csv') # for cities proper

In [678]:
df_msa = pd.read_csv('data/cbsa-est2024-alldata-char.csv') # for msas

df_msa = df_msa[df_msa['SUMLEV'] != 314]

df_sf = df_msa[(df_msa['NAME'].str.contains('San Francisco')) & (df_msa['AGEGRP'] == 0) & (df_msa['YEAR'] == 6)].copy()
df_over5 = df_msa[(df_msa['TOT_POP'] > 5000000) & (df_msa['AGEGRP'] == 0) & (df_msa['YEAR'] == 6)].copy()

df_sf_5yr = df_msa[(df_msa['NAME'].str.contains('San Francisco')) & (df_msa['AGEGRP'] == 0) & (df_msa['YEAR'] == 1)]
df_over5_5yr = df_msa[(df_msa['AGEGRP'] == 0) & (df_msa['YEAR'] == 1)] # No population requirement here because some cities grew bigger

df_sf['4 year population change'] = df_sf['TOT_POP'].values- df_sf_5yr['TOT_POP'].values
df_sf['4 year population change pct'] = (df_sf['TOT_POP'].values/df_sf_5yr['TOT_POP'].values - 1)


df_over5 = df_over5.merge(
    df_over5_5yr[['CBSA', 'TOT_POP']],
    on='CBSA',
    suffixes=('', '_2020')
)

# Calculate changes
df_over5['4 year population change']      = df_over5['TOT_POP'] - df_over5['TOT_POP_2020']
df_over5['4 year population change pct']  = (df_over5['TOT_POP'] / df_over5['TOT_POP_2020'] - 1)

# Clean up
df_over5 = df_over5.drop(columns=['TOT_POP_2020'])

In [679]:
msa_sample = pd.concat([df_sf, df_over5])

In [680]:
msa_sample = msa_sample.drop(['SUMLEV', 'CBSA', 'MDIV','YEAR', 'AGEGRP','WA_MALE', 'WA_FEMALE', 'BA_MALE',
       'BA_FEMALE', 'IA_MALE', 'IA_FEMALE', 'AA_MALE', 'AA_FEMALE', 'NA_MALE',
       'NA_FEMALE', 'TOM_MALE', 'TOM_FEMALE', 'WAC_MALE', 'WAC_FEMALE',
       'BAC_MALE', 'BAC_FEMALE', 'IAC_MALE', 'IAC_FEMALE', 'AAC_MALE',
       'AAC_FEMALE', 'NAC_MALE', 'NAC_FEMALE', 'NH_MALE', 'NH_FEMALE',
       'NHWA_MALE', 'NHWA_FEMALE', 'NHBA_MALE', 'NHBA_FEMALE', 'NHIA_MALE',
       'NHIA_FEMALE', 'NHAA_MALE', 'NHAA_FEMALE', 'NHNA_MALE', 'NHNA_FEMALE',
       'NHTOM_MALE', 'NHTOM_FEMALE', 'NHWAC_MALE', 'NHWAC_FEMALE',
       'NHBAC_MALE', 'NHBAC_FEMALE', 'NHIAC_MALE', 'NHIAC_FEMALE',
       'NHAAC_MALE', 'NHAAC_FEMALE', 'NHNAC_MALE', 'NHNAC_FEMALE', 'H_MALE',
       'H_FEMALE', 'HWA_MALE', 'HWA_FEMALE', 'HBA_MALE', 'HBA_FEMALE',
       'HIA_MALE', 'HIA_FEMALE', 'HAA_MALE', 'HAA_FEMALE', 'HNA_MALE',
       'HNA_FEMALE', 'HTOM_MALE', 'HTOM_FEMALE', 'HWAC_MALE', 'HWAC_FEMALE',
       'HBAC_MALE', 'HBAC_FEMALE', 'HIAC_MALE', 'HIAC_FEMALE', 'HAAC_MALE',
       'HAAC_FEMALE', 'HNAC_MALE', 'HNAC_FEMALE'], axis = 1)
msa_sample = msa_sample.reset_index(drop=True)

In [681]:
msa_first_city_state = pd.Series([
    'San Francisco, CA',
 'New York, NY',
 'Los Angeles, CA',
 'Chicago, IL',
 'Dallas, TX',
 'Houston, TX',
 'Miami, FL',
 'Washington, DC',
 'Atlanta, GA',
 'Philadelphia, PA',
 'Phoenix, AZ',
 'Boston, MA'
])
msa_sample['common_name'] = msa_first_city_state

In [682]:
msa_sample.head(24)

,NAME,LSAD,TOT_POP,TOT_MALE,TOT_FEMALE,4 year population change,4 year population change pct,common_name
0,"San Francisco-Oakland-Fremont, CA",Metropolitan Statistical Area,4648486,2315027,2333459,-105169,-0.02,"San Francisco, CA"
1,"New York-Newark-Jersey City, NY-NJ",Metropolitan Statistical Area,19940274,9722142,10218132,-143138,-0.01,"New York, NY"
2,"Los Angeles-Long Beach-Anaheim, CA",Metropolitan Statistical Area,12927614,6394633,6532981,-277043,-0.02,"Los Angeles, CA"
3,"Chicago-Naperville-Elgin, IL-IN",Metropolitan Statistical Area,9408576,4631974,4776602,-45787,-0.00,"Chicago, IL"
4,"Dallas-Fort Worth-Arlington, TX",Metropolitan Statistical Area,8344032,4134816,4209216,705774,0.09,"Dallas, TX"
5,"Houston-Pasadena-The Woodlands, TX",Metropolitan Statistical Area,7796182,3867729,3928453,645949,0.09,"Houston, TX"
6,"Miami-Fort Lauderdale-West Palm Beach, FL",Metropolitan Statistical Area,6457988,3170380,3287608,319631,0.05,"Miami, FL"
7,"Washington-Arlington-Alexandria, DC-VA-MD-WV",Metropolitan Statistical Area,6436489,3163503,3272986,157894,0.03,"Washington, DC"
8,"Atlanta-Sandy Springs-Roswell, GA",Metropolitan Statistical Area,6411149,3109052,3302097,304344,0.05,"Atlanta, GA"
9,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",Metropolitan Statistical Area,6330422,3074073,3256349,85405,0.01,"Philadelphia, PA"


Zillow housing data sourced from https://www.zillow.com/research/data/

In [683]:
df_zhvi =pd.read_csv('data/Metro_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv')
df_zhvi = df_zhvi[df_zhvi['RegionType'] == 'msa']
df_zhvi = df_zhvi[['RegionName', '2025-10-31', '2020-10-31']]
df_zhvi['5yr_change_pct'] = df_zhvi['2025-10-31'].div(df_zhvi['2020-10-31']).sub(1)
df_zhvi = df_zhvi.drop(['2025-10-31'], axis=1)
df_zhvi.columns = ['MSA', 'Median Home Sale Price', 'Median Home Sale Price 5 yr Percent Change']
df_zhvi

,MSA,Median Home Sale Price,Median Home Sale Price 5 yr Percent Change
1,"New York, NY",521377.36,0.35
2,"Los Angeles, CA",696346.75,0.35
3,"Chicago, IL",246367.61,0.37
4,"Dallas, TX",270056.74,0.34
5,"Houston, TX",234190.85,0.30
...,...,...,...
890,"Zapata, TX",157317.93,-0.21
891,"Ketchikan, AK",318502.04,0.21
892,"Craig, CO",199913.17,0.51
893,"Vernon, TX",99983.81,-0.10


In [684]:
msa_sample = msa_sample.merge(df_zhvi, how='left', left_on='common_name', right_on = 'MSA')

In [685]:
msa_sample

,NAME,LSAD,TOT_POP,TOT_MALE,TOT_FEMALE,4 year population change,4 year population change pct,common_name,MSA,Median Home Sale Price,Median Home Sale Price 5 yr Percent Change
0,"San Francisco-Oakland-Fremont, CA",Metropolitan Statistical Area,4648486,2315027,2333459,-105169,-0.02,"San Francisco, CA","San Francisco, CA",955368.08,0.15
1,"New York-Newark-Jersey City, NY-NJ",Metropolitan Statistical Area,19940274,9722142,10218132,-143138,-0.01,"New York, NY","New York, NY",521377.36,0.35
2,"Los Angeles-Long Beach-Anaheim, CA",Metropolitan Statistical Area,12927614,6394633,6532981,-277043,-0.02,"Los Angeles, CA","Los Angeles, CA",696346.75,0.35
3,"Chicago-Naperville-Elgin, IL-IN",Metropolitan Statistical Area,9408576,4631974,4776602,-45787,-0.00,"Chicago, IL","Chicago, IL",246367.61,0.37
4,"Dallas-Fort Worth-Arlington, TX",Metropolitan Statistical Area,8344032,4134816,4209216,705774,0.09,"Dallas, TX","Dallas, TX",270056.74,0.34
5,"Houston-Pasadena-The Woodlands, TX",Metropolitan Statistical Area,7796182,3867729,3928453,645949,0.09,"Houston, TX","Houston, TX",234190.85,0.30
6,"Miami-Fort Lauderdale-West Palm Beach, FL",Metropolitan Statistical Area,6457988,3170380,3287608,319631,0.05,"Miami, FL","Miami, FL",315264.79,0.49
7,"Washington-Arlington-Alexandria, DC-VA-MD-WV",Metropolitan Statistical Area,6436489,3163503,3272986,157894,0.03,"Washington, DC","Washington, DC",456231.48,0.25
8,"Atlanta-Sandy Springs-Roswell, GA",Metropolitan Statistical Area,6411149,3109052,3302097,304344,0.05,"Atlanta, GA","Atlanta, GA",264193.35,0.43
9,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",Metropolitan Statistical Area,6330422,3074073,3256349,85405,0.01,"Philadelphia, PA","Philadelphia, PA",266146.18,0.41


In [686]:
df_zori = pd.read_csv('data/Metro_zori_uc_sfrcondomfr_sm_month.csv')
df_zori = df_zori[df_zori['RegionType'] == 'msa']
df_zori = df_zori[['RegionName', '2025-10-31', '2020-10-31']]
df_zori['5yr_change_pct'] = df_zori['2025-10-31'].div(df_zori['2020-10-31']).sub(1)
df_zori = df_zori.drop(['2025-10-31'], axis=1)
df_zori.columns = ['MSA', 'Median Rent', 'Median Rent 5 yr Percent Change']
df_zori

,MSA,Median Rent,Median Rent 5 yr Percent Change
1,"New York, NY",2356.14,0.44
2,"Los Angeles, CA",2254.08,0.30
3,"Chicago, IL",1511.54,0.37
4,"Dallas, TX",1336.73,0.26
5,"Houston, TX",1347.95,0.22
...,...,...,...
702,"Sterling, CO",NaN,NaN
703,"Jamestown, ND",NaN,NaN
704,"Portales, NM",NaN,NaN
705,"Los Alamos, NM",NaN,NaN


In [687]:
msa_sample = msa_sample.merge(df_zori, how='left', left_on='common_name', right_on = 'MSA')

In [688]:
msa_sample = msa_sample.drop(['LSAD', 'common_name', 'MSA_x', 'MSA_y'], axis = 1)

In [689]:
msa_sample.to_csv('data/us_city_affordability.csv')
msa_sample

,NAME,TOT_POP,TOT_MALE,TOT_FEMALE,4 year population change,4 year population change pct,Median Home Sale Price,Median Home Sale Price 5 yr Percent Change,Median Rent,Median Rent 5 yr Percent Change
0,"San Francisco-Oakland-Fremont, CA",4648486,2315027,2333459,-105169,-0.02,955368.08,0.15,2584.07,0.21
1,"New York-Newark-Jersey City, NY-NJ",19940274,9722142,10218132,-143138,-0.01,521377.36,0.35,2356.14,0.44
2,"Los Angeles-Long Beach-Anaheim, CA",12927614,6394633,6532981,-277043,-0.02,696346.75,0.35,2254.08,0.30
3,"Chicago-Naperville-Elgin, IL-IN",9408576,4631974,4776602,-45787,-0.00,246367.61,0.37,1511.54,0.37
4,"Dallas-Fort Worth-Arlington, TX",8344032,4134816,4209216,705774,0.09,270056.74,0.34,1336.73,0.26
5,"Houston-Pasadena-The Woodlands, TX",7796182,3867729,3928453,645949,0.09,234190.85,0.30,1347.95,0.22
6,"Miami-Fort Lauderdale-West Palm Beach, FL",6457988,3170380,3287608,319631,0.05,315264.79,0.49,1752.49,0.52
7,"Washington-Arlington-Alexandria, DC-VA-MD-WV",6436489,3163503,3272986,157894,0.03,456231.48,0.25,1885.58,0.26
8,"Atlanta-Sandy Springs-Roswell, GA",6411149,3109052,3302097,304344,0.05,264193.35,0.43,1410.32,0.32
9,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",6330422,3074073,3256349,85405,0.01,266146.18,0.41,1441.08,0.30


# City-Level Analysis
Processing city-level census data with city-level Zillow housing data

Note that we use census sumlev = 162 for city proper.

Per https://www2.census.gov/programs-surveys/popest/technical-documentation/file-layouts/2020-2024/SUB-EST2024.pdf SUMLEV 162 corresponds to an incorporated place and most of the entities we are interested in appear to hold this designation.

In [703]:
# Filter df_sub for cities proper (SUMLEV = 162 for cities)
df_cities = df_sub[df_sub['SUMLEV'] == 162].copy()

# Define the same subset of cities as used in MSA analysis
target_cities = [
    'San Francisco',
    'New York',
    'Los Angeles',
    'Chicago',
    'Dallas',
    'Houston',
    'Miami',
    'Washington',
    'Atlanta',
    'Philadelphia',
    'Phoenix',
    'Boston'
]

# Filter for target cities (remove 'city' suffix and clean names)
df_cities['CITY_NAME'] = df_cities['NAME'].str.replace(' city', '').str.replace(' (pt.)', '')
city_sample = df_cities[df_cities['CITY_NAME'].isin(target_cities)].copy()

# Calculate population changes
city_sample['4 year population change'] = city_sample['POPESTIMATE2024'] - city_sample['POPESTIMATE2020']
city_sample['4 year population change pct'] = (city_sample['POPESTIMATE2024'] - city_sample['POPESTIMATE2020']) / city_sample['POPESTIMATE2020']

# Create common names for merging with Zillow data
city_common_names = {
    'San Francisco': 'San Francisco, CA',
    'New York': 'New York, NY',
    'Los Angeles': 'Los Angeles, CA',
    'Chicago': 'Chicago, IL',
    'Dallas': 'Dallas, TX',
    'Houston': 'Houston, TX',
    'Miami': 'Miami, FL',
    'Washington': 'Washington, DC',
    'Atlanta': 'Atlanta, GA',
    'Philadelphia': 'Philadelphia, PA',
    'Phoenix': 'Phoenix, AZ',
    'Boston': 'Boston, MA'
}

city_sample['common_name'] = city_sample['CITY_NAME'].map(city_common_names)
city_sample = city_sample[city_sample['common_name'].notna()]
city_sample['TOT_POP'] = city_sample['POPESTIMATE2024']
# Add city and state columns for proper joining
city_sample['city'] = city_sample['CITY_NAME']
# Convert numeric state codes to state abbreviations to match Zillow data
state_abbr = {
    1: 'AL', 2: 'AK', 4: 'AZ', 5: 'AR', 6: 'CA', 8: 'CO', 9: 'CT', 10: 'DE', 11: 'DC', 12: 'FL',
    13: 'GA', 15: 'HI', 16: 'ID', 17: 'IL', 18: 'IN', 19: 'IA', 20: 'KS', 21: 'KY', 22: 'LA', 23: 'ME',
    24: 'MD', 25: 'MA', 26: 'MI', 27: 'MN', 28: 'MS', 29: 'MO', 30: 'MT', 31: 'NE', 32: 'NV', 33: 'NH',
    34: 'NJ', 35: 'NM', 36: 'NY', 37: 'NC', 38: 'ND', 39: 'OH', 40: 'OK', 41: 'OR', 42: 'PA', 44: 'RI',
    45: 'SC', 46: 'SD', 47: 'TN', 48: 'TX', 49: 'UT', 50: 'VT', 51: 'VA', 53: 'WA', 54: 'WV', 55: 'WI', 56: 'WY'
}
city_sample['state'] = city_sample['STATE'].map(state_abbr)
city_sample['state'] = city_sample['state'].str.upper()

In [704]:
df_cities

,SUMLEV,STATE,COUNTY,PLACE,COUSUB,CONCIT,PRIMGEO_FLAG,FUNCSTAT,NAME,STNAME,ESTIMATESBASE2020,POPESTIMATE2020,POPESTIMATE2021,POPESTIMATE2022,POPESTIMATE2023,POPESTIMATE2024,CITY_NAME


In [691]:
city_sample = city_sample[city_sample['TOT_POP'] > 50000] # Get rid of Miami, UT and the like
city_sample

,SUMLEV,STATE,COUNTY,PLACE,COUSUB,CONCIT,PRIMGEO_FLAG,FUNCSTAT,NAME,STNAME,...,POPESTIMATE2022,POPESTIMATE2023,POPESTIMATE2024,CITY_NAME,4 year population change,4 year population change pct,common_name,TOT_POP,city,state


In [693]:
city_sample['common_name'] = city_sample['common_name'].str.replace(',.*', '', regex=True)

In [694]:
df_zhvi_city = pd.read_csv('data/City_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv')
df_zhvi_city = df_zhvi_city[df_zhvi_city['RegionType'] == 'city']
df_zhvi_city = df_zhvi_city[['RegionName', 'State', '2025-10-31', '2020-10-31']]
df_zhvi_city['5yr_change_pct'] = df_zhvi_city['2025-10-31'].div(df_zhvi_city['2020-10-31']).sub(1)
df_zhvi_city = df_zhvi_city.drop(['2025-10-31'], axis=1)
df_zhvi_city.columns = ['city', 'state', 'Median Home Sale Price', 'Median Home Sale Price 5 yr Percent Change']

In [695]:
df_zori_city = pd.read_csv('data/City_zori_uc_sfrcondomfr_sm_month.csv')
df_zori_city = df_zori_city[df_zori_city['RegionType'] == 'city']
df_zori_city = df_zori_city[['RegionName', 'State', '2025-10-31', '2020-10-31']]
df_zori_city['5yr_change_pct'] = df_zori_city['2025-10-31'].div(df_zori_city['2020-10-31']).sub(1)
df_zori_city = df_zori_city.drop(['2025-10-31'], axis=1)
df_zori_city.columns = ['city', 'state', 'Median Rent', 'Median Rent 5 yr Percent Change']

Rent values aren't available for DC and Boston for some reason

In [696]:
df_zori_city[df_zori_city['city'] == 'Washington']

,city,state,Median Rent,Median Rent 5 yr Percent Change
27,Washington,DC,NaN,NaN
1231,Washington,PA,NaN,NaN
2155,Washington,UT,NaN,NaN
2238,Washington,MI,NaN,NaN
2530,Washington,MO,NaN,NaN
3160,Washington,NJ,NaN,NaN


In [697]:
df_zori_city[df_zori_city['city'] == 'Boston']

,city,state,Median Rent,Median Rent 5 yr Percent Change
29,Boston,MA,NaN,NaN


In [698]:
# Merge city sample with housing data using both city and state to avoid duplications
city_sample = city_sample.merge(df_zhvi_city, how='left', left_on=['city', 'state'], right_on=['city', 'state'])
city_sample = city_sample.merge(df_zori_city, how='left', left_on=['city', 'state'], right_on=['city', 'state'])

# Clean up duplicate columns
city_sample = city_sample.drop(['city_x', 'state_x', 'city_y', 'state_y'], axis=1, errors='ignore')
city_sample = city_sample.drop_duplicates(subset=['CITY_NAME'], keep='first')

In [699]:
city_sample['NAME'] = city_sample['NAME'].str.replace(' city', '', regex=True)

In [700]:
# Save city-level results with proper joins (no duplications)
city_sample = city_sample[['NAME', 'TOT_POP', '4 year population change', '4 year population change pct', 
           'Median Home Sale Price', 'Median Home Sale Price 5 yr Percent Change',
           'Median Rent', 'Median Rent 5 yr Percent Change']]
city_sample.to_csv('data/us_city_affordability_city_level.csv', index=False)


In [702]:
city_sample

,NAME,TOT_POP,4 year population change,4 year population change pct,Median Home Sale Price,Median Home Sale Price 5 yr Percent Change,Median Rent,Median Rent 5 yr Percent Change
